In [2]:
import io
import lzma
import os
import re
import subprocess

from Bio.Data.IUPACData import protein_letters
import Bio.SeqIO

from IPython.display import display, HTML

import matplotlib.pyplot as plt

import pandas as pd

from plotnine import *

import yaml

In [3]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

In [4]:
os.makedirs(config['gisaid_mutations_dir'], exist_ok=True)

In [5]:
spikes = list(Bio.SeqIO.parse('data/spike_test.fasta', 'fasta')) 

In [6]:
print(f"Read {len(spikes)} spike sequences.")

Read 28 spike sequences.


In [ ]:
#spikes_df = (
#    pd.DataFrame({'seqrecord': spikes})
#    .assign(description=lambda x: x['seqrecord'].map(lambda rec: rec.description),
#            country=lambda x: x['description'].str.split('|').str[-1],
#            host=lambda x: x['description'].str.split('|').str[6].str.strip(),
#            length=lambda x: x['seqrecord'].map(len),
#            n_ambiguous=lambda x: x['seqrecord'].map(lambda rec: rec.seq.count('X') + rec.seq.count('x')),
#            )
#    )

In [42]:
spikes_df =pd.DataFrame({'seqrecord': spikes}).assign(description=lambda x: x['seqrecord'].map(lambda rec: rec.description),
                                              country=lambda x: x['description'].str.split('/').str[1],
                                               host='Human', #lambda x: x['description'].str.split('/').str[0].str.strip(),
                                               length=lambda x: x['seqrecord'].map(len),
                                              n_ambiguous=lambda x: x['seqrecord'].map(lambda rec: rec.seq.count('X') + rec.seq.count('x')))

In [44]:
spikes_df

,seqrecord,description,country,host,length,n_ambiguous
0,"(G, T, T, A, A, T, C, T, T, A, C, A, A, C, C, ...",hCoV-19/Thailand/64086728__2019/2021 extraction,Thailand,Human,3869,0
1,"(G, T, T, A, A, T, C, T, T, A, C, A, A, C, C, ...",hCoV-19/Botswana/BOT441877/2021 extraction,Botswana,Human,3869,0
2,"(A, C, T, A, G, T, C, T, C, T, A, G, T, C, A, ...",hCoV-19/USA/AR-PED-0195/2021 extraction,USA,Human,3869,0
3,"(G, T, T, A, A, T, C, T, T, A, C, A, A, C, C, ...",hCoV-19/France/PDL-CHUN-00098/2021 extraction,France,Human,3869,0
4,"(A, G, T, C, T, C, T, A, G, T, C, A, T, T, G, ...",hCoV-19/Ecuador/NIC-INSPI-52906/2021 extraction,Ecuador,Human,3869,0
5,"(T, T, T, T, A, T, T, G, C, C, A, C, T, A, G, ...",hCoV-19/USA/NM-NCGR-2021109263_Consensus/2021 ...,USA,Human,3869,0
6,"(A, A, A, C, G, A, A, C, A, A, T, G, T, T, T, ...",hCoV-19/Gabon/W0742/2021 extraction,Gabon,Human,3869,0
7,"(T, G, T, T, A, A, T, T, T, T, A, C, A, A, C, ...",hCoV-19/SouthAfrica/NHLS-UCT-GS-B975/2021 extr...,SouthAfrica,Human,3869,0
8,"(G, T, T, A, A, T, C, T, T, A, C, A, A, C, C, ...",hCoV-19/Canada/UN-89237/2021 extraction,Canada,Human,3869,0
9,"(G, T, T, A, A, T, C, T, T, A, C, A, A, C, C, ...",hCoV-19/France/PDL-CHUN-00125/2021 extraction,France,Human,3869,0


In [45]:
display(HTML(
    spikes_df
    .groupby('host')
    .aggregate(n_sequences=pd.NamedAgg('seqrecord', 'count'))
    .sort_values('n_sequences', ascending=False)
    .to_html()
    ))

spikes_df = spikes_df.query('host == "Human"')

,n_sequences
host,
Human,28
